In [28]:
import numpy as np

def show_integer_properties():
    Table = "{:<6} {:<22} {:<22} {:<22}"
    print(Table.format("Bytes","Largest Unsigned Int","Minimum Signed Int","Maximum Signed Int"))
    
    for ii in range(1, 9):
        int_largest_unsigned_int = 2**(8*ii)-1
        int_minimum_signed_int = -(int_largest_unsigned_int+1 >> 1)
        int_maximum_signed_int = (int_largest_unsigned_int-1) >> 1
        print(Table.format(str(ii), str(int_largest_unsigned_int), str(int_minimum_signed_int), str(int_maximum_signed_int)))

In [29]:
show_integer_properties()

Bytes  Largest Unsigned Int   Minimum Signed Int     Maximum Signed Int    
1      255                    -128                   127                   
2      65535                  -32768                 32767                 
3      16777215               -8388608               8388607               
4      4294967295             -2147483648            2147483647            
5      1099511627775          -549755813888          549755813887          
6      281474976710655        -140737488355328       140737488355327       
7      72057594037927935      -36028797018963968     36028797018963967     
8      18446744073709551615   -9223372036854775808   9223372036854775807   


In [88]:
import numpy
m = numpy.array([1],dtype= numpy.int16)

m.size()

TypeError: 'int' object is not callable

In [96]:
import numpy as np
from time import time

def estimate_wrap_around():
    
    m = np.array([1], dtype=np.int16)
    seconds_start = time()
    while m[0] > 0:
        m[0] += 1
    seconds_runtime = time() - seconds_start
    str_line1 = "measured 16-bit time (microseconds): " + str(seconds_runtime * 1e6)
    
    str_line0 = "estimated 8-bit time (nanoseconds): " + str(seconds_runtime * 2**(8-16) * 1e9)
    str_line2 = "estimated 32-bit time (seconds): " + str(seconds_runtime * 2**(32-16))
    str_line3 = "estimated 64-bit time (years): " + str(seconds_runtime * 2**(64-16) /60/60/365)
    
    print(str_line0)
    print(str_line1)
    print(str_line2)
    print(str_line3)
    
estimate_wrap_around()

estimated 8-bit time (nanoseconds): 804670.1550483704
measured 16-bit time (microseconds): 205995.5596923828
estimated 32-bit time (seconds): 13500.125
estimated 64-bit time (years): 44126784.90632573


In [89]:
2**16 / 2**8

256.0

In [92]:
674 / 256 * 1000

2632.8125

In [98]:
import numpy as np
from time import time

def estimate_wrap_around():
    m = np.array([1], dtype=np.int8)
    start = time()
    while (m[0] << 0) > 0:
        m[0] += 1
    print("estimated 8-bit time (nanoseconds):", (time()-start)*1e9)
    
    m = np.array([1], dtype=np.int16)
    start = time()
    while (m[0] << 0) > 0:
        m[0] += 1
    print("measured 16-bit time (microseconds):", (time()-start)*1e6)
    
estimate_wrap_around()

def estimate_wrap_around():
    m = np.array([1], dtype=np.int8)
    start = time()
    while (m[0] << 1) > 0:
        m[0] += 1
    print("estimated 8-bit time (nanoseconds):", (time()-start)*2e9)
    
    m = np.array([1], dtype=np.int16)
    start = time()
    while (m[0] << 1) > 0:
        m[0] += 1
    print("measured 16-bit time (microseconds):", (time()-start)*2e6)
    
estimate_wrap_around()

estimated 8-bit time (nanoseconds): 1000165.9393310547
measured 16-bit time (microseconds): 200034.85679626465
estimated 8-bit time (nanoseconds): 1980304.7180175781
measured 16-bit time (microseconds): 663982.8681945801


In [35]:
digits = []
for i in range(0, 11):
    digits.append(1/(2**i))
digits

[1.0,
 0.5,
 0.25,
 0.125,
 0.0625,
 0.03125,
 0.015625,
 0.0078125,
 0.00390625,
 0.001953125,
 0.0009765625]

In [72]:
def is_precise_float16(arg):
    
    num_fraction_bits = 10
    e_max = 15
    
    digits = []
    for i in range(0, num_fraction_bits+1):
        digits.append(1/(2**i))
    
    exponent = 0
    while(arg > 1):
        arg /= 2
        exponent += 1
#     print("exponent is", exponent)
    if exponent > e_max:
        return False
    
    fraction = 0
    for digit in digits:
        if arg >= digit:
#             print(digit)
            arg -= digit
            fraction += digit
#     print("fraction is", fraction)
            
#     print(fraction * (2**exponent))
            
    return (arg == 0)

In [77]:
test_num = 1
result = True
while(result == True):
#     print("test_num is", test_num)
    result = is_precise_float16(test_num)
    test_num += 1
#     print("")
test_num-1

1025

In [86]:
a = numpy.float16(1025)

In [88]:
a - 1025

0.0

In [78]:
def is_precise_float32(arg):
    
    num_fraction_bits = 23
    e_max = 127
    
    digits = []
    for i in range(0, num_fraction_bits+1):
        digits.append(1/(2**i))
    
    exponent = 0
    while(arg > 1):
        arg /= 2
        exponent += 1
#     print("exponent is", exponent)
    if exponent > e_max:
        return False
    
    fraction = 0
    for digit in digits:
        if arg >= digit:
#             print(digit)
            arg -= digit
            fraction += digit
#     print("fraction is", fraction)
            
#     print(fraction * (2**exponent))
            
    return (arg == 0)

In [92]:
is_precise_float32(2**23+1)

False

In [80]:
test_num = 1
result = True
while(result == True):
#     print("test_num is", test_num)
    result = is_precise_float32(test_num)
    test_num += 1
#     print("")
test_num-1

KeyboardInterrupt: 

In [32]:
barray = b'\x01\x02\x03'
b_little = int.from_bytes(barray, byteorder="little")
b_big = int.from_bytes(barray, byteorder="big")

print(b_little)
print(b_big)

0x010203, 0x030201

197121
66051


(66051, 197121)

In [ ]:
# 2.5 is 2097152 fraction and 128 exponent
# binary to float 32 representation
x = 0b0100_0000_0010_0000_0000_0000_0000_0000

print(x.to_bytes(length=4, ))

In [118]:
0b0111_1011_1111_1111
byte_array = b'\x07\x0b\x0f\x0f'
int.from_bytes(byte_array, byteorder="little")

252644103

In [152]:
# a = numpy.float16(67)
# b = a.tobytes()
b = b'\x07\x0b\x0f\x0f'
c = numpy.frombuffer(b, dtype=numpy.float16)


b'\xff{'

In [163]:
a = numpy.float16(1027)
b = a.tobytes()
c = numpy.frombuffer(b, dtype=numpy.float16)
a, b, c

(1027.0, b'\x03d', array([1027.], dtype=float16))